In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

sns.set_palette("Set3")

In [2]:
df = pd.read_csv("Top_Anime_data.csv")

<h3>-- Inspection of the dataset --</h3>

In [3]:
#Get info of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Score        1000 non-null   float64
 1   Popularity   1000 non-null   int64  
 2   Rank         1000 non-null   int64  
 3   Members      1000 non-null   int64  
 4   Description  1000 non-null   object 
 5   Synonyms     709 non-null    object 
 6   Japanese     999 non-null    object 
 7   English      859 non-null    object 
 8   Type         1000 non-null   object 
 9   Episodes     1000 non-null   object 
 10  Status       1000 non-null   object 
 11  Aired        1000 non-null   object 
 12  Premiered    569 non-null    object 
 13  Broadcast    569 non-null    object 
 14  Producers    1000 non-null   object 
 15  Licensors    1000 non-null   object 
 16  Studios      1000 non-null   object 
 17  Source       1000 non-null   object 
 18  Genres       771 non-null    object 
 19  Demogra

In [4]:
df.head()

,Score,Popularity,Rank,Members,Description,Synonyms,Japanese,English,Type,Episodes,...,Premiered,Broadcast,Producers,Licensors,Studios,Source,Genres,Demographic,Duration,Rating
0,9.38,284,1,710,During their decade-long quest to defeat the D...,Frieren at the Funeral,葬送のフリーレン,Frieren: Beyond Journey's End,TV,28,...,Fall 2023,Fridays at 23:00 (JST),"Aniplex, Dentsu, Shogakukan-Shueisha Productio...","None found, add some",Madhouse,Manga,"AdventureAdventure, DramaDrama, FantasyFantasy",ShounenShounen,24 min. per ep.,PG-13 - Teens 13 or older
1,9.09,3,2,3,After a horrific alchemy experiment goes wrong...,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist: Brotherhood,TV,64,...,Spring 2009,Sundays at 17:00 (JST),"Aniplex, Square Enix, Mainichi Broadcasting Sy...","Funimation, Aniplex of America",Bones,Manga,"ActionAction, AdventureAdventure, DramaDrama, ...",ShounenShounen,24 min. per ep.,R - 17+ (violence & profanity)
2,9.07,13,3,2,Eccentric scientist Rintarou Okabe has a never...,NaN,STEINS;GATE,Steins;Gate,TV,24,...,Spring 2011,Wednesdays at 02:05 (JST),"Frontier Works, Media Factory, Kadokawa Shoten...",Funimation,White Fox,Visual novel,"DramaDrama, Sci-FiSci-Fi, SuspenseSuspense",NaN,24 min. per ep.,PG-13 - Teens 13 or older
3,9.06,342,4,630,"Gintoki, Shinpachi, and Kagura return as the f...",Gintama' (2015),銀魂°,Gintama Season 4,TV,51,...,Spring 2015,Wednesdays at 18:00 (JST),"TV Tokyo, Aniplex, Dentsu","Funimation, Crunchyroll",Bandai Namco Pictures,Manga,"ActionAction, ComedyComedy, Sci-FiSci-Fi",ShounenShounen,24 min. per ep.,PG-13 - Teens 13 or older
4,9.05,21,5,2,Seeking to restore humanity's diminishing hope...,NaN,進撃の巨人 Season3 Part.2,Attack on Titan Season 3 Part 2,TV,10,...,Spring 2019,Mondays at 00:10 (JST),"Production I.G, Dentsu, Mainichi Broadcasting ...",Funimation,Wit Studio,Manga,"ActionAction, DramaDrama, SuspenseSuspense",ShounenShounen,23 min. per ep.,R - 17+ (violence & profanity)


In [5]:
#Convert names to snake_Case to avoid empty spaces of upper case letters:
df.columns = [columns.lower().replace(" ", "_") for columns in df.columns]
df.columns

Index(['score', 'popularity', 'rank', 'members', 'description', 'synonyms',
       'japanese', 'english', 'type', 'episodes', 'status', 'aired',
       'premiered', 'broadcast', 'producers', 'licensors', 'studios', 'source',
       'genres', 'demographic', 'duration', 'rating'],
      dtype='object')

In [6]:
# Determine which columns are numerical and which are categorical
df_num = df.select_dtypes(include="number")
num_columns = list(df_num.columns)
print(f"The numerical columns are :{num_columns}")

df_cat = df.select_dtypes(exclude="number")
cat_columns = list(df_cat.columns)
print(f"The categorical columns are :{cat_columns}")

The numerical columns are :['score', 'popularity', 'rank', 'members']
The categorical columns are :['description', 'synonyms', 'japanese', 'english', 'type', 'episodes', 'status', 'aired', 'premiered', 'broadcast', 'producers', 'licensors', 'studios', 'source', 'genres', 'demographic', 'duration', 'rating']


In [7]:
#Function to check null values and duplicates
def null_duplicate_check(df, df_name):
    RED = "\033[91m"
    RESET = "\033[0m"

    # Nulls
    non_null = df.notna().sum()
    nulls = df.isna().sum()
    na_percentage = (df.isna().mean() * 100).round(2)

    # Duplicates
    dup_total = df.duplicated()
    dup_rows = df[dup_total].index.astype(str)

    print(f"""
🔍 Checking {df_name}

Number of non-null values:
{non_null}

Number of null values:
{RED}{nulls}{RESET}

Percentage of null values:
{na_percentage}
""")

    if dup_total.any():
        print(f"⚠ Duplicate rows: {', '.join(dup_rows)}")
    else:
        print("✔ No duplicate rows found")

    cols_with_na = nulls[nulls > 0].index
    if len(cols_with_na) > 0:
        print(f"⚠ Columns with null values: {', '.join(cols_with_na)}")
    else:
        print("✔ No null values in columns")

    print("-" * 50)


In [8]:
#Checking numerical data
null_duplicate_check(df_num, "Numerical Data")


🔍 Checking Numerical Data

Number of non-null values:
score         1000
popularity    1000
rank          1000
members       1000
dtype: int64

Number of null values:
score         0
popularity    0
rank          0
members       0
dtype: int64

Percentage of null values:
score         0.0
popularity    0.0
rank          0.0
members       0.0
dtype: float64

✔ No duplicate rows found
✔ No null values in columns
--------------------------------------------------


In [9]:
#Checking categorical data
null_duplicate_check(df_cat, "Categorical Data")


🔍 Checking Categorical Data

Number of non-null values:
description    1000
synonyms        709
japanese        999
english         859
type           1000
episodes       1000
status         1000
aired          1000
premiered       569
broadcast       569
producers      1000
licensors      1000
studios        1000
source         1000
genres          771
demographic     521
duration       1000
rating         1000
dtype: int64

Number of null values:
description      0
synonyms       291
japanese         1
english        141
type             0
episodes         0
status           0
aired            0
premiered      431
broadcast      431
producers        0
licensors        0
studios          0
source           0
genres         229
demographic    479
duration         0
rating           0
dtype: int64

Percentage of null values:
description     0.0
synonyms       29.1
japanese        0.1
english        14.1
type            0.0
episodes        0.0
status          0.0
aired           0.0
pre

The only column with null values that will have impact is "genres", so we will work on it later

In [10]:
#Working with categorical data
#Delete spaces from begginig or end
cat_cols_str=df_cat.columns
df_cat[cat_cols_str]= df_cat[cat_cols_str].apply(lambda col: col.str.strip())

In [11]:
#Function to check whitespaces
def check_whitespace_all_strings(df):
    RED = "\033[91m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RESET = "\033[0m"

    str_cols = df_cat.select_dtypes(include=["object", "string"]).columns

    if len(str_cols) == 0:
        print(f"{GREEN}✔ No string columns found{RESET}")
        return

    print(f"\n🔍 Checking whitespace in {len(str_cols)} string columns\n")

    issues_found = False

    for col in str_cols:
        series = df_cat[col]

        leading_trailing = series.str.contains(r"^\s+|\s+$", na=False).sum()
        only_spaces = series.str.fullmatch(r"\s+", na=False).sum()

        if leading_trailing > 0 or only_spaces > 0:
            issues_found = True
            print(f"{RED}⚠ Column: {col}{RESET}")
            print(f"{YELLOW}  - Leading/trailing spaces: {leading_trailing}{RESET}")
            print(f"{YELLOW}  - Values with only spaces: {only_spaces}{RESET}")
            print()

    if not issues_found:
        print(f"{GREEN}✔ All string columns are clean (no leading/trailing spaces){RESET}")


In [12]:
check_whitespace_all_strings(df_cat)


🔍 Checking whitespace in 18 string columns

✔ All string columns are clean (no leading/trailing spaces)


<h3>-- Data Formatting --</h3>

We want to refine the data on some columns to get more insights or fixing data problems
Columns affected: episodes, aired, studios and genres

<h4>Episodes</h4>
Some animes have Unknown numbers of episodes, so we want to make them numbers to work with

In [13]:
#Making episodes column numeric
#All null values set to zero, the zeros can be dropped later if needed.
df["episodes"] = pd.to_numeric(df["episodes"], errors = "coerce").fillna(0).astype(int)

In [14]:
# We check what animes are affected
df.loc[df["episodes"] == 0, ["english"]]


,english
50,One Piece
389,Holo Graffiti
419,Case Closed


<h4>Aired</h4>
It has format: begginig to end. We will split them to work with both data
The ? values will be replaced with the date of the extraction of the database (Dec 31, 2024)

*We also have to look for movies wich only have one date*

In [15]:
# Split aired in two
dates = df["aired"].str.split(" to ", n=1, expand=True)
df["start_date"] = pd.to_datetime(dates[0], errors="coerce")
df["end_date"] = pd.to_datetime(dates[1], errors="coerce")

# Deal with movies. Start date = End date
movies = dates[1].isna()
df.loc[movies,"end_date"] = df.loc[movies,"start_date"]

# Deal with "to ?" results
cut_date = "Dec 31, 2024"
ongoing = dates[1].eq("?")
df.loc[ongoing, "end_date"] = cut_date

df["total_aired"] = (df["end_date"] - df["start_date"]).dt.days / 365

In [16]:
# df[["english","start_date","end_date"]].head(51)
# This confirms that 'One Piece' has now the end date that we want

<h4>Studios</h4>
We will convert the strings in lists

In [17]:
df["studios"]

0                   Madhouse
1                      Bones
2                  White Fox
3      Bandai Namco Pictures
4                 Wit Studio
               ...          
995          Satelight, 8bit
996                 Lay-duce
997        TMS Entertainment
998        TMS Entertainment
999     SANZIGEN, LIDENFILMS
Name: studios, Length: 1000, dtype: object

*We found "None found" and "add some" as data for the studios. Which we will have to remove*

In [18]:
remove = {"None found", "add some"}

df["studios"] = (
    df["studios"]
    .str.split(",")
    .apply(lambda lst: [
        s.strip()
        for s in lst
        if s.strip().lower() not in remove
    ])
)


In [19]:
### example way can be used to find indexes that contain a studio

# mask = df["studios"].apply(lambda _: "Madhouse" in _)        
# df_madhouse = df[mask]

# madhouse_production_count =  df_madhouse.shape[0]
# madhouse_production_count

In [20]:
df["studios"]

0                   [Madhouse]
1                      [Bones]
2                  [White Fox]
3      [Bandai Namco Pictures]
4                 [Wit Studio]
                ...           
995          [Satelight, 8bit]
996                 [Lay-duce]
997        [TMS Entertainment]
998        [TMS Entertainment]
999     [SANZIGEN, LIDENFILMS]
Name: studios, Length: 1000, dtype: object

We have to store only the names of the studios once.

In [21]:
list_unique_studios = set().union(*df["studios"])
list_unique_studios
# We use an empty set and union (to check all previuos lists) with df[*studios] (* unpacks studios, so each list is acted on not the series),

{'8bit',
 'A-1 Pictures',
 'A.C.G.T.',
 'ABJ COMPANY',
 'AHA Entertainment',
 'AIC',
 'AIC ASTA',
 'APPP',
 'Actas',
 'Ajia-do',
 'Animation Do',
 'Artland',
 'B.CMAY PICTURES',
 'BUG FILMS',
 'Bandai Namco Pictures',
 'Bee Train',
 'Bibury Animation Studios',
 'Bones',
 "Brain's Base",
 'C-Station',
 'C2C',
 'CLAP',
 'Cloud Hearts',
 'CloverWorks',
 'CoMix Wave Films',
 'Colored Pencil Animation',
 'CygamesPictures',
 'DandeLion Animation Studio',
 'David Production',
 'Doga Kobo',
 'Drive',
 'E&G Films',
 'Egg Firm',
 'Encourage Films',
 'Fenz',
 'Flat Studio',
 'Gainax',
 'Gallop',
 'Geek Toys',
 'Geno Studio',
 'GoHands',
 'Gonzo',
 'Graphinica',
 'Group TAC',
 'HAL Film Maker',
 'HMCH',
 'HORNETS',
 'Haoliners Animation League',
 'Hololive Production',
 'Hoods Entertainment',
 'Imagin',
 'J.C.Staff',
 'K-Factory',
 'Khara',
 'Kinema Citrus',
 'Kitty Film Mitaka Studio',
 'Kyoto Animation',
 'LAN Studio',
 'LIDENFILMS',
 'Lapin Track',
 'Lay-duce',
 'Lerche',
 'M.S.C',
 'MAPPA',
 '

<h4>Genres</h4>
We have words duplicated so we will have to split them into a list: genres_list.

In [22]:
def remove_duplicated_string(s):                # Function to remove duplicated words after checking if its really duplicated.
    if pd.isna(s):
        return pd.NA   
                 
    mid = len(s) // 2
    if s[:mid] == s[mid:]:
        return s[:mid]
    return s

def clean_genres_cell(cell):                    # check if cell is NaN
    if pd.isna(cell):
        return pd.NA                            # returns a "clean" NA

    
    genres = str(cell).split(",")               # Converts to a string and separates by comma

    cleaned = []

    for g in genres:                            # Clean each individual value
        g = g.strip()                           # remove whitespaces
        g = remove_duplicated_string(g)         # uses previous function to remove duplicates
        cleaned.append(g)

    return ", ".join(cleaned)                   # Joins in the new string

Now we have to assing each  cleaned genre to a new column to be able to make analysis later

In [23]:
df["genres"] = df["genres"].apply(clean_genres_cell)
df["genres"]

0              Adventure, Drama, Fantasy
1      Action, Adventure, Drama, Fantasy
2                Drama, Sci-Fi, Suspense
3                 Action, Comedy, Sci-Fi
4                Action, Drama, Suspense
                     ...                
995              Action, Romance, Sci-Fi
996           Action, Adventure, Fantasy
997           Adventure, Comedy, Mystery
998           Adventure, Comedy, Mystery
999                                 <NA>
Name: genres, Length: 1000, dtype: object

In [24]:
df["genres_list"] = df["genres"].str.split(", ")        # From string to list

genres_pivot = (                                      
    df["genres_list"]
    .explode()                                          # Convert list to rows
    .str.get_dummies()                                  # Convert unique values in columns
    .groupby(level=0).sum()                             # Groups again for original row and adds
)

df = df.join(genres_pivot)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 45 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   score          1000 non-null   float64       
 1   popularity     1000 non-null   int64         
 2   rank           1000 non-null   int64         
 3   members        1000 non-null   int64         
 4   description    1000 non-null   object        
 5   synonyms       709 non-null    object        
 6   japanese       999 non-null    object        
 7   english        859 non-null    object        
 8   type           1000 non-null   object        
 9   episodes       1000 non-null   int64         
 10  status         1000 non-null   object        
 11  aired          1000 non-null   object        
 12  premiered      569 non-null    object        
 13  broadcast      569 non-null    object        
 14  producers      1000 non-null   object        
 15  licensors      1000 no

<h4>English vs Japanese Names</h4>
Some English names have null values, so we will replace them with the japanese names

In [26]:
df["name"] = df["english"].fillna(df["japanese"])

<h4>Duration</h4>
Data has stored the duration of TV animes as "X min. per ep." while movies as "X hr. Y min." so we will calculate the total duration

In [27]:
def to_minutes(d):
    if pd.isna(d):
        return pd.NA

    d = d.lower().replace(".", "")

    total = 0

    if "hr" in d:
        h = int(d.split("hr")[0].strip())
        total += 60 * h
        d = d.split("hr", 1)[1]

    if "min" in d:
        m = int(d.split("min")[0].strip())
        total += m

    return total


df["duration_min"] = df["duration"].apply(to_minutes)


Now we calculate the total runtime of each anime (duration in minutes * number of episodes)

In [28]:
df["total_runtime"] = df["duration_min"] * df["episodes"]
df[["duration_min","episodes","total_runtime"]]

,duration_min,episodes,total_runtime
0,24,28,672
1,24,64,1536
2,24,24,576
3,24,51,1224
4,23,10,230
...,...,...,...
995,119,1,119
996,24,5,120
997,110,1,110
998,116,1,116


<h4>End of Modifications</h4>
Once the modifications on the desired columns has ended. We will do a new inspection of the dataset

In [29]:
#Convert names to snake_case to avoid empty spaces of upper case letters: New columns were added
df.columns = [columns.lower().replace(" ", "_") for columns in df.columns]
df.columns

Index(['score', 'popularity', 'rank', 'members', 'description', 'synonyms',
       'japanese', 'english', 'type', 'episodes', 'status', 'aired',
       'premiered', 'broadcast', 'producers', 'licensors', 'studios', 'source',
       'genres', 'demographic', 'duration', 'rating', 'start_date', 'end_date',
       'total_aired', 'genres_list', 'action', 'adventure', 'avant_garde',
       'award_winning', 'boys_love', 'comedy', 'drama', 'ecchi', 'fantasy',
       'girls_love', 'gourmet', 'horror', 'mystery', 'romance', 'sci-fi',
       'slice_of_life', 'sports', 'supernatural', 'suspense', 'name',
       'duration_min', 'total_runtime'],
      dtype='object')

In [30]:
df_num = df.select_dtypes(include="number")
num_columns = list(df_num.columns)
print(f"The numerical columns are :{num_columns}")

df_cat = df.select_dtypes(exclude="number")
cat_columns = list(df_cat.columns)
print(f"The categorical columns are :{cat_columns}")

The numerical columns are :['score', 'popularity', 'rank', 'members', 'episodes', 'total_aired', 'action', 'adventure', 'avant_garde', 'award_winning', 'boys_love', 'comedy', 'drama', 'ecchi', 'fantasy', 'girls_love', 'gourmet', 'horror', 'mystery', 'romance', 'sci-fi', 'slice_of_life', 'sports', 'supernatural', 'suspense', 'duration_min', 'total_runtime']
The categorical columns are :['description', 'synonyms', 'japanese', 'english', 'type', 'status', 'aired', 'premiered', 'broadcast', 'producers', 'licensors', 'studios', 'source', 'genres', 'demographic', 'duration', 'rating', 'start_date', 'end_date', 'genres_list', 'name']


In [31]:
null_duplicate_check(df_num, "Numerical Data")


🔍 Checking Numerical Data

Number of non-null values:
score            1000
popularity       1000
rank             1000
members          1000
episodes         1000
total_aired      1000
action           1000
adventure        1000
avant_garde      1000
award_winning    1000
boys_love        1000
comedy           1000
drama            1000
ecchi            1000
fantasy          1000
girls_love       1000
gourmet          1000
horror           1000
mystery          1000
romance          1000
sci-fi           1000
slice_of_life    1000
sports           1000
supernatural     1000
suspense         1000
duration_min     1000
total_runtime    1000
dtype: int64

Number of null values:
score            0
popularity       0
rank             0
members          0
episodes         0
total_aired      0
action           0
adventure        0
avant_garde      0
award_winning    0
boys_love        0
comedy           0
drama            0
ecchi            0
fantasy          0
girls_love       0
gourmet   

In [32]:
# Check which columns contain lists, so the test could fail
cols_with_lists = df.columns[df.map(lambda x: isinstance(x, list)).any()]
cols_with_lists

Index(['studios', 'genres_list'], dtype='object')

In [33]:
df_cat_no_lists = df_cat.drop(columns=["studios", "genres_list"])    
null_duplicate_check(df_cat_no_lists, "Categorical Data")


🔍 Checking Categorical Data

Number of non-null values:
description    1000
synonyms        709
japanese        999
english         859
type           1000
status         1000
aired          1000
premiered       569
broadcast       569
producers      1000
licensors      1000
source         1000
genres          771
demographic     521
duration       1000
rating         1000
start_date     1000
end_date       1000
name            999
dtype: int64

Number of null values:
description      0
synonyms       291
japanese         1
english        141
type             0
status           0
aired            0
premiered      431
broadcast      431
producers        0
licensors        0
source           0
genres         229
demographic    479
duration         0
rating           0
start_date       0
end_date         0
name             1
dtype: int64

Percentage of null values:
description     0.0
synonyms       29.1
japanese        0.1
english        14.1
type            0.0
status          0.0
aire

In [34]:
check_whitespace_all_strings(df_cat)


🔍 Checking whitespace in 19 string columns

⚠ Column: description
  - Leading/trailing spaces: 128
  - Values with only spaces: 0



<h3>-- Prepare new dataset --</h3>
Dropping columns that are not going to be used and reordering the most important ones

In [35]:
num_drop = ['popularity', 'members']
cat_drop = ['description', 'synonyms', 'japanese', 'status', 'aired', 'premiered', 'broadcast', 'producers', 'licensors', 'demographic', 'english', 'duration']
df_drop = num_drop + cat_drop

anime = df.drop(columns = df_drop)

In [36]:
anime.head()

,score,rank,type,episodes,studios,source,genres,rating,start_date,end_date,...,mystery,romance,sci-fi,slice_of_life,sports,supernatural,suspense,name,duration_min,total_runtime
0,9.38,1,TV,28,[Madhouse],Manga,"Adventure, Drama, Fantasy",PG-13 - Teens 13 or older,2023-09-29,2024-03-22,...,0,0,0,0,0,0,0,Frieren: Beyond Journey's End,24,672
1,9.09,2,TV,64,[Bones],Manga,"Action, Adventure, Drama, Fantasy",R - 17+ (violence & profanity),2009-04-05,2010-07-04,...,0,0,0,0,0,0,0,Fullmetal Alchemist: Brotherhood,24,1536
2,9.07,3,TV,24,[White Fox],Visual novel,"Drama, Sci-Fi, Suspense",PG-13 - Teens 13 or older,2011-04-06,2011-09-14,...,0,0,1,0,0,0,1,Steins;Gate,24,576
3,9.06,4,TV,51,[Bandai Namco Pictures],Manga,"Action, Comedy, Sci-Fi",PG-13 - Teens 13 or older,2015-04-08,2016-03-30,...,0,0,1,0,0,0,0,Gintama Season 4,24,1224
4,9.05,5,TV,10,[Wit Studio],Manga,"Action, Drama, Suspense",R - 17+ (violence & profanity),2019-04-29,2019-07-01,...,0,0,0,0,0,0,1,Attack on Titan Season 3 Part 2,23,230


In [37]:
anime.columns

Index(['score', 'rank', 'type', 'episodes', 'studios', 'source', 'genres',
       'rating', 'start_date', 'end_date', 'total_aired', 'genres_list',
       'action', 'adventure', 'avant_garde', 'award_winning', 'boys_love',
       'comedy', 'drama', 'ecchi', 'fantasy', 'girls_love', 'gourmet',
       'horror', 'mystery', 'romance', 'sci-fi', 'slice_of_life', 'sports',
       'supernatural', 'suspense', 'name', 'duration_min', 'total_runtime'],
      dtype='object')

In [38]:
cols = ["rank", "score", "name", "episodes", "duration_min", "total_runtime"]
anime = anime[cols + [c for c in anime.columns if c not in cols]]


In [39]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   rank           1000 non-null   int64         
 1   score          1000 non-null   float64       
 2   name           999 non-null    object        
 3   episodes       1000 non-null   int64         
 4   duration_min   1000 non-null   int64         
 5   total_runtime  1000 non-null   int64         
 6   type           1000 non-null   object        
 7   studios        1000 non-null   object        
 8   source         1000 non-null   object        
 9   genres         771 non-null    object        
 10  rating         1000 non-null   object        
 11  start_date     1000 non-null   datetime64[ns]
 12  end_date       1000 non-null   datetime64[ns]
 13  total_aired    1000 non-null   float64       
 14  genres_list    771 non-null    object        
 15  action         1000 no

In [40]:
anime.to_csv('anime_clean.csv', index=False)   # Save to .csv in case we need to check it from outside pandas